In [1]:
import math
import torch

# use a GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.float


**Read Data and convert them into tensor readable format**

In [2]:
import pandas as pd
import numpy as np
df=pd.read_csv("tuning.csv")

po=df["power"]
ti=df["time"]
p=po
t=ti
global change
def change(df):
    df=np.array(df)
    df=np.reshape(df,(-1,1))
    df=torch.FloatTensor(df)
    return df
tx=change(po)
ty=change(ti)
lst=np.zeros((len(po),2))
for i in range(len(po)):
    lst[i][0]=tx[i]
    lst[i][1]=ty[i]
lst=torch.FloatTensor(lst)

**Fit Surrogate**

In [3]:
from botorch.models import SingleTaskGP
from gpytorch.constraints import GreaterThan


model = SingleTaskGP(train_X=lst, train_Y=ty)
model.likelihood.noise_covar.register_constraint("raw_noise", GreaterThan(1e-5))

In [4]:
from gpytorch.mlls import ExactMarginalLogLikelihood

mll = ExactMarginalLogLikelihood(likelihood=model.likelihood, model=model)
# set mll and all submodules to the specified dtype and device
mll = mll.to(tx)

In [5]:
from torch.optim import SGD

optimizer = SGD([{'params': model.parameters()}], lr=0.1)

In [24]:
model.eval();

**Create Acquisition functions**

**(1)Expected Improvement**

In [7]:
from botorch.acquisition import ExpectedImprovement
best_value = ty.max()
EI = ExpectedImprovement(model=model, best_f=best_value)

In [8]:
from botorch.optim import optimize_acqf

new_point_analytic, _ = optimize_acqf(
    acq_function=EI,
    bounds=torch.tensor([[500.0,500.0], [5000.0,20000.0]]),
    q=1,
    num_restarts=20,
    raw_samples=100,
    options={},
)

In [9]:
print(new_point_analytic)

tensor([[4774.3613, 3249.7761]])


**(2)Monte Carlo acquisition function**

In [49]:
from botorch.sampling import SobolQMCNormalSampler
from botorch.acquisition import qExpectedImprovement
sampler = SobolQMCNormalSampler(num_samples=500, seed=0, resample=False)        
MC_EI = qExpectedImprovement(
    model, best_f=best_value, sampler=sampler
)
torch.manual_seed(seed=0) # to keep the restart conditions the same
new_point_mc, _ = optimize_acqf(
    acq_function=MC_EI,
    bounds=torch.tensor([[500.0,500.0], [5000.0,20000.0]]),
    q=1,
    num_restarts=20,
    raw_samples=100,
    options={},
)

In [50]:
print("MC recommended power and time : ",new_point_mc[0],"\nEI  recommended power and time: ",new_point_analytic)

MC recommended power:  tensor([4928.0376, 8899.7021]) 
EI  recommended power:  tensor([[4910.6167, 4499.9937]])
